In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,RANSACRegressor,TheilSenRegressor
from statsmodels.robust.robust_linear_model import RLM
import statsmodels
import pandas as pd
from tqdm import tqdm_notebook
import random
from scipy import optimize
from sklearn.covariance import MinCovDet as MCD
import time

In [2]:
main_size = 300

In [3]:
def Add_Column(df, n = main_size):
    return np.column_stack([np.ones([df.shape[0], 1], dtype=np.int32), df])

In [4]:
ideal_coefs = np.array([5,2])

In [63]:
def MC(df, y):
    sum_mm = []
    sum_hbr = []
    sum_mmr = []
    sum_hbrr = []
    nu = 1
    for i in tqdm_notebook(range(30)):   
        #МНК
        lr = LinearRegression(fit_intercept=False)
        lr.fit(df_norm, y_norm)
        mnk_coefs = lr.coef_
        #ransac
        ransac = RANSACRegressor(lr)
        ransac.fit(df_norm, y_norm)
        ransac_coefs = ransac.estimator_.coef_
        #LMS-оценка
        lms = LMS(df_norm, y_norm)
        ## ММ-оценка
        mm_est = MM(df_norm, y_norm, lms)
        #HBR-оценка
        hbr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, lms), disp = 0)
        hbr_est = np.array([np.median(y_norm - hbr_beta * df_norm[:,1]),hbr_beta])
        #HBRR-оценка
        hbrr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, ransac_coefs), disp = 0)
        hbrr_est = np.array([np.median(y_norm - hbrr_beta * df_norm[:,1]),hbrr_beta])
        #MMR-оценка
        mmr_est = MM(df_norm, y_norm, ransac_coefs)
        #подсчет
        
        sum_mm.append(np.linalg.norm(ideal_coefs - mm_est))
        sum_hbr.append(np.linalg.norm(ideal_coefs - hbr_est))
        sum_mmr.append(np.linalg.norm(ideal_coefs - mmr_est))
        sum_hbrr.append(np.linalg.norm(ideal_coefs - hbrr_est))
        
    return sum_mm, sum_mmr, sum_hbr, sum_hbrr

In [64]:
def normal(mean, var, sz = main_size):
    return st.norm.rvs(loc = mean, scale = np.sqrt(var), size = sz)

def unif(a,b):
    return st.uniform.rvs(loc = a, scale = b-a, size = main_size)
    
def tukey(var_1, var_2, bp = 0.1, sz = main_size):
    eps = np.array([])
    for i in range(sz):
        c = st.uniform.rvs()
        if c >= bp:
            eps = np.append(eps, st.norm.rvs(scale = np.sqrt(var_1)))
        else:
            eps = np.append(eps, st.norm.rvs(scale = np.sqrt(var_2)))
    return eps
            
def cauchy(sz = main_size):
    return st.t.rvs(df = 1, size = sz)

In [65]:
def LMS(df, y):
    h = int(df.shape[0]/2 + 1)
    Q = 1000000000
    beta = np.array([1,1],dtype=float)
    for i in range(df.shape[0]):
        for j in range(i):
            if df[:,1][i] != df[:,1][j]:
                beta[1] = (y[i]-y[j])/(df[:,1][i] - df[:,1][j])
                beta[0] = df[:,1][i]*y[j] - df[:,1][j]*y[i]
                med = np.median(np.abs(y - np.sum(beta*df, axis=1)))
                if med < Q:
                    Q = med.copy()
                    ans = beta.copy()
    return ans

In [66]:
def MM(df, y, blms):
    n = df.shape[0]
    s = y - np.sum(blms * df, axis = 1)
    res_itog = np.abs(np.median(s) - s)
    MADN = np.median(np.abs(res_itog)) / 0.675
    const1 = 10
    const2 = 4.68
    var = s/(const1*MADN)
    w = np.minimum(3 - 3*np.power(var,2) + np.power(var,4), 1/np.power(var,2))
    sigma_1 = np.dot(w,np.power(s/const1,2))
    sigma_1 = np.sqrt(sigma_1 * 2/ df.shape[0])
    eps = 0.0001
    while(np.abs(sigma_1/MADN - 1) > eps):
        MADN = sigma_1
        sigma_1 = 0
        w = np.minimum(3 - 3*np.power(var,2) + np.power(var,4), 1/np.power(var,2))
        sigma_1 = np.dot(w,np.power(s/const1,2))
        sigma_1 = np.sqrt(sigma_1 * 2 / n)
    var2 = s/(const2*MADN)
    w = np.minimum(3 - 3*np.power(var2,2) + np.power(var2,4), 1/np.power(var2,2))
    W = np.diag(w)
    beta = blms
    beta_1 = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(df.T,W), df)), df.T), W), y)
    eps = 0.001
    while np.linalg.norm(beta_1-beta)/np.linalg.norm(beta) > eps:
        s = y - np.sum(beta_1 * df, axis = 1)
        var2 = s/(const2*sigma_1)
        w = np.minimum(3 - 3*np.power(var2,2) + np.power(var2,4), 1/np.power(var2,2))
        W = np.diag(w)
        beta = beta_1
        beta_1 = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(df.T,W), df)), df.T), W), y)
    return beta_1

In [67]:
def R(x, df, y):
    s = 0
    array = y - x*df
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    return np.dot(array, ranks/(df.shape[0]) - 0.5)

In [68]:
def HBR(x, df, y, lts):
    resid = y - np.sum(lts * df,axis = 1)
    MADN = 1.483*np.median(np.abs(resid - np.median(resid)))
    mcd = MCD()
    mcd.fit(df[:,1].reshape(-1,1))
    mahal =  mcd.mahalanobis(df[:,1].reshape(-1,1))
    b, c = 4, 4 
    resid = np.abs(resid / MADN)
    resid_new = y - x * df[:,1]
    resid_dif = np.abs(resid_new - resid_new.reshape(-1,1))
    resid_dif = np.tril(resid_dif, -1)
    mahal = b / mahal
    mahal[mahal > 1] = 1
    mahal = mahal / resid
    mahal = mahal.reshape(-1, 1) * mahal
    mahal = np.tril(mahal, -1)
    mahal = c * mahal
    mahal[mahal > 1] = 1
    ans = np.sum(mahal * resid_dif)
    return ans

In [70]:
for perc in [1, 5, 20, 30, 40]:
    percent = int(perc * main_size / 100)
    df_norm = normal(20, 5)
    eps = normal(0, 3)
    y_norm = 5 + 2*df_norm + eps
    y_norm[:percent] += 500
    df_norm = Add_Column(df_norm)
    sum_mm, sum_mmr, sum_hbr, sum_hbrr = MC(df_norm, y_norm)
    print('percent = {} %, sum_mm = {:0.2f}, sum_mmr = {:0.2f}, sum_hbr = {:0.2f}, sum_hbrr = {:0.2f}'.format(
                                               perc, 
                                               np.mean(sum_mm), 
                                               np.mean(sum_mmr), 
                                               np.mean(sum_hbr), 
                                               np.mean(sum_hbrr)))

percent = 1 %, sum_mm = 0.80, sum_mmr = 0.81, sum_hbr = 0.01, sum_hbrr = 0.03


percent = 5 %, sum_mm = 1.52, sum_mmr = 1.51, sum_hbr = 1.41, sum_hbrr = 1.41


percent = 20 %, sum_mm = 1.29, sum_mmr = 1.29, sum_hbr = 1.89, sum_hbrr = 1.84


percent = 30 %, sum_mm = 0.67, sum_mmr = 0.67, sum_hbr = 1.41, sum_hbrr = 1.40


percent = 40 %, sum_mm = 0.50, sum_mmr = 0.50, sum_hbr = 0.54, sum_hbrr = 0.39
